# Exercise: Data Quality

## Auditing Data Quality

In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up. In the first exercise we want you to audit
the datatypes that can be found in some particular fields in the dataset.
The possible types of values can be:
- NoneType if the value is a string "NULL" or an empty string ""
- list, if the value starts with "{"
- int, if the value can be cast to int
- float, if the value can be cast to float, but CANNOT be cast to int.
   For example, '3.23e+07' should be considered a float because it can be cast
   as float but int('3.23e+07') will throw a ValueError
- 'str', for all other values

The audit_file function should return a dictionary containing fieldnames and a 
SET of the types that can be found in the field. e.g.
{"field1": set([type(float()), type(int()), type(str())]),
 "field2": set([type(str())]),
  ....
}
The type() function returns a type object describing the argument given to the 
function. You can also use examples of objects to create type objects, e.g.
type(1.1) for a float: see the test function below for examples.

Note that the first three rows (after the header row) in the cities.csv file
are not actual data points. The contents of these rows should not be included
when processing data types. Be sure to include functionality in your code to
skip over or detect these rows.


In [3]:
import codecs
import csv
import json
import pprint

CITIES = 'data/cities.csv'

FIELDS = ["name", "timeZone_label", "utcOffset", "homepage", "governmentType_label",
          "isPartOf_label", "areaCode", "populationTotal", "elevation",
          "maximumElevation", "minimumElevation", "populationDensity",
          "wgs84_pos#lat", "wgs84_pos#long", "areaLand", "areaMetro", "areaUrban"]

def audit_file(filename, fields):
    fieldtypes = {}
    for field in fields:
        fieldtypes[field] = set()
    
    with open(filename, 'r') as f:
        reader = csv.DictReader(f)
        
        for row in reader:
            if (row['URI']=='URI') or (row['URI'].startswith('http://www.w3.org')):
                continue
                
            for field in fields:
                value = row[field]               
                try:
                    value = int(value)   
                    valuetype = type(value)
                except:
                    try:
                        value = float(value)
                        valuetype = type(value)
                    except:
                        if value in ['NULL', '']:
                            valuetype = type(None)
                        elif value.startswith('{'):
                            valuetype = type([])
                        else:
                            valuetype = type('')
                    
                fieldtypes[field].add(valuetype)          
                
    return fieldtypes


fieldtypes = audit_file(CITIES, FIELDS)

pprint.pprint(fieldtypes)


{'areaCode': {<class 'str'>, <class 'int'>, <class 'NoneType'>, <class 'list'>},
 'areaLand': {<class 'NoneType'>, <class 'list'>, <class 'float'>},
 'areaMetro': {<class 'NoneType'>, <class 'list'>, <class 'float'>},
 'areaUrban': {<class 'NoneType'>, <class 'list'>, <class 'float'>},
 'elevation': {<class 'NoneType'>, <class 'list'>, <class 'float'>},
 'governmentType_label': {<class 'str'>, <class 'NoneType'>, <class 'list'>},
 'homepage': {<class 'str'>, <class 'NoneType'>, <class 'list'>},
 'isPartOf_label': {<class 'str'>, <class 'NoneType'>, <class 'list'>},
 'maximumElevation': {<class 'NoneType'>, <class 'list'>, <class 'float'>},
 'minimumElevation': {<class 'NoneType'>, <class 'float'>},
 'name': {<class 'str'>, <class 'NoneType'>, <class 'list'>},
 'populationDensity': {<class 'NoneType'>, <class 'list'>, <class 'float'>},
 'populationTotal': {<class 'int'>, <class 'NoneType'>, <class 'list'>},
 'timeZone_label': {<class 'str'>, <class 'NoneType'>, <class 'list'>},
 'utcOff

## Fixing the area

In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.

Since in the previous quiz you made a decision on which value to keep for the
"areaLand" field, you now know what has to be done.

Finish the function fix_area(). It will receive a string as an input, and it
has to return a float representing the value of the area or None.
You have to change the function fix_area. You can use extra functions if you
like, but changes to process_file will not be taken into account.
The rest of the code is just an example on how this function can be used.


In [4]:
CITIES = 'data/cities.csv'


def fix_area(area):

    try:
        area = float(area)
    except:
        if area.startswith('{'):
            arealist = area[1:-1].split('|')
            arealens = [len(s) for s in arealist]
            index = arealens.index(max(arealens))
            area = float(arealist[index])    
            
        else:
            area = None

    return area



def process_file(filename):
    data = []

    with open(filename, "r") as f:
        reader = csv.DictReader(f)

        #skipping the extra metadata
        for i in range(3):
            next(reader)

        # processing file
        for line in reader:
            # calling your function to fix the area value
            if "areaLand" in line:
                line["areaLand"] = fix_area(line["areaLand"])
            data.append(line)

    return data



data = process_file(CITIES)

print("Printing three example results:")
for n in range(5,8):
    pprint.pprint(data[n]["areaLand"])



Printing three example results:
None
None
None


## Fixing name

In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.

In the previous quiz you recognized that the "name" value can be an array (or
list in Python terms). It would make it easier to process and query the data
later if all values for the name are in a Python list, instead of being
just a string separated with special characters, like now.

Finish the function fix_name(). It will recieve a string as an input, and it
will return a list of all the names. If there is only one name, the list will
have only one item in it; if the name is "NULL", the list should be empty.
The rest of the code is just an example on how this function can be used.

In [6]:
CITIES = 'data/cities.csv'


def fix_name(name):

    if name == 'NULL':
        name = []
    elif name.startswith('{'):
        name = name[1:-1].split('|')
    else:
        name = [name]
        
    return name


def process_file(filename):
    data = []
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        
        #skipping the extra metadata
        for i in range(3):
            next(reader)
            
        # processing file
        for line in reader:
            # calling your function to fix the area value
            if "name" in line:
                line["name"] = fix_name(line["name"])
            data.append(line)
    return data



data = process_file(CITIES)

print("Printing 20 results:")
for n in range(20):
    pprint.pprint(data[n]["name"])


Printing 20 results:
['Kud']
['Kuju']
['Kumbhraj']
['Kumhari']
['Kundagola', 'Kundgol ಕುಂದಗೋಳ']
['Kunigal']
['Kunzer']
['Kurduvadi', 'कुर्डुवाडी']
['Kurgunta']
['Kurinjipadi']
['Kurud']
['Kushtagi']
['Ladnun', 'लाडनूँ']
['Lahar', 'लहार']
['Laharpur']
['Lakheri']
['Lakhipur']
['Laksar', 'लक्सर']
['Lalkuan']
['Lalsot']


## Crossfield Auditing

In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.

If you look at the full city data, you will notice that there are couple of
values that seem to provide the same information in different formats: "point"
seems to be the combination of "wgs84_pos#lat" and "wgs84_pos#long". However,
we do not know if that is the case and should check if they are equivalent.

Finish the function check_loc(). It will recieve 3 strings: first, the combined
value of "point" followed by the separate "wgs84_pos#" values. You have to
extract the lat and long values from the "point" argument and compare them to
the "wgs84_pos# values, returning True or False.

Note that you do not have to fix the values, only determine if they are
consistent. To fix them in this case you would need more information. Feel free
to discuss possible strategies for fixing this on the discussion forum.

The rest of the code is just an example on how this function can be used.
Changes to "process_file" function will not be taken into account for grading.

In [12]:
CITIES = 'data/cities.csv'


def check_loc(point, lat, longi):
    
    if (point=='NULL') or (point.startswith('{')):
        return False
    else:
        point_lat, point_longi = point.split(' ') 
        point_lat = float(point_lat)
        point_longi = float(point_longi)
        lat = float(lat)
        longi = float(longi)
    
        return (point_lat == lat) and (point_longi == longi)
    
    


def process_file(filename):
    data = []
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        #skipping the extra matadata
        for i in range(3):
            next(reader)
        # processing file
        for line in reader:
            # calling your function to check the location
            result = check_loc(line["point"], line["wgs84_pos#lat"], line["wgs84_pos#long"])
            if not result:
                print("{}: {} != {} {}".
                      format(line["name"], line["point"], line["wgs84_pos#lat"], line["wgs84_pos#long"]))
            data.append(line)

    return data


check_loc("33.08 75.28", "33.08", "75.28") == True

True